In [ ]:
import pandas as pd
import json
from collections import defaultdict

def merge_dta_files(json_path, output_excel_path, data_folder_path, chunk_size=100):
    """
    Reads a JSON file for merging instructions, processes DTA files in chunks,
    merges them based on person and household IDs, and saves the result to an Excel file.

    Args:
        json_path (str): The file path for the JSON file with merging instructions.
        output_excel_path (str): The file path for the output Excel file.
        data_folder_path (str): The path to the folder containing the DTA files.
        chunk_size (int): The number of rows to process from the initial DTA file.
    """
    try:
        with open(json_path, 'r') as f:
            config = json.load(f)
    except FileNotFoundError:
        print(f"Error: JSON file not found at '{json_path}'")
        return
    except json.JSONDecodeError:
        print(f"Error: Could not decode JSON from '{json_path}'")
        return

    # Define merging keys from the JSON file
    person_keys = ['caseid', 'v001', 'v002', 'v003']
    household_keys = ['hhid', 'v001', 'v002'] 


    # Group features by their source file for efficient processing
    features_by_file = defaultdict(lambda: {'person': [], 'household': []})
    for feature in config.get('useful', []):
        source_file = feature.get('source_file')
        feature_name = feature.get('feature_name')
        merge_on = feature.get('merge_on')

        if source_file and feature_name:
            if merge_on == 'person_id':
                features_by_file[source_file]['person'].append(feature_name)
            elif merge_on == 'household_id':
                features_by_file[source_file]['household'].append(feature_name)

    # Identify the primary file to start the merge process
    # This is typically the file with the most person-level features
    primary_file = max(features_by_file, key=lambda f: len(features_by_file[f]['person']))
    
    # Start with the first chunk of the primary file
    try:
        reader = pd.read_stata(f"{data_folder_path}/{primary_file}", chunksize=chunk_size, convert_categoricals=False)
        merged_df = next(reader)
    except FileNotFoundError:
        print(f"Error: Primary DTA file not found at '{data_folder_path}/{primary_file}'")
        return
    except Exception as e:
        print(f"An error occurred while reading '{primary_file}': {e}")
        return

    print(f"Successfully read initial chunk from '{primary_file}'.")

    # Sequentially merge features from each file
    for file, features in features_by_file.items():
        if file == primary_file:
            continue

        try:
            # Read the corresponding chunk from the other files
            other_reader = pd.read_stata(f"{data_folder_path}/{file}", chunksize=chunk_size, convert_categoricals=False)
            other_df = next(other_reader)

            # Merge person-level features
            if features['person']:
                cols_to_merge = person_keys + features['person']
                merged_df = pd.merge(merged_df, other_df[cols_to_merge], on=person_keys, how='left')
                print(f"Merged person features from '{file}'.")

            # Merge household-level features
            if features['household']:
                # Ensure household keys in the file are correctly identified
                current_household_keys = [k for k in household_keys if k in other_df.columns]
                cols_to_merge = current_household_keys + features['household']
                
                # Check if all household keys are present for merging
                if all(k in merged_df.columns for k in current_household_keys):
                    merged_df = pd.merge(merged_df, other_df[cols_to_merge], on=current_household_keys, how='left')
                    print(f"Merged household features from '{file}'.")
                else:
                    print(f"Warning: Could not merge household data from '{file}' due to missing keys.")

        except FileNotFoundError:
            print(f"Warning: DTA file not found at '{data_folder_path}/{file}'. Skipping.")
        except StopIteration:
            print(f"Warning: No more data in '{file}' to merge. Skipping.")
        except Exception as e:
            print(f"An error occurred while processing '{file}': {e}")

    # Save the final merged dataframe to an Excel file
    try:
        merged_df.to_excel(output_excel_path, index=False)
        print(f"\n✅ Successfully created merged file at '{output_excel_path}'.")
    except Exception as e:
        print(f"An error occurred while saving the Excel file: {e}")

if __name__ == '__main__':
    # --- Configuration ---
    # Path to the JSON file with merging instructions
    JSON_FILE_PATH = 'household_features_400_200.json'
    
    # Path for the final merged Excel file
    OUTPUT_EXCEL_PATH = 'merged_household_data.xlsx'
    
    # Folder containing all the DTA files
    # Create a folder named 'Dataset' and place all your .dta files inside it.
    DATA_FOLDER_PATH = 'Dataset' 
    
    # --- Execution ---
    merge_dta_files(JSON_FILE_PATH, OUTPUT_EXCEL_PATH, DATA_FOLDER_PATH)


Successfully read initial chunk from 'IAKR7EFL.DTA'.
An error occurred while processing 'IAMR7EFL.DTA': "None of [Index(['caseid', 'v001', 'v002', 'v003', 'age', 'daily_calorie_intake',\n       'meals_per_day', 'platelet_count_thousands', 'triglycerides_mg_dl',\n       'urinary_arsenic_ug_l', 'weight_kg', 'respiratory_symptoms',\n       'immunization_status', 'tsh_uIU_ml (lab)',\n       'urinary_potassium_mmol_l (lab)', 'transferrin_mg_dl (lab)',\n       'urinary_sodium_mmol_l (lab)', 'serum_albumin_g_dl (lab)',\n       'tsh_uIU_ml (lab)', 'urinary_potassium_mmol_l (lab)',\n       'transferrin_mg_dl (lab)', 'urinary_sodium_mmol_l (lab)',\n       'serum_albumin_g_dl (lab)', 'tsh_uIU_ml (lab)',\n       'urinary_potassium_mmol_l (lab)', 'transferrin_mg_dl (lab)',\n       'urinary_sodium_mmol_l (lab)', 'serum_albumin_g_dl (lab)',\n       'tsh_uIU_ml (lab)', 'urinary_potassium_mmol_l (lab)',\n       'transferrin_mg_dl (lab)', 'urinary_sodium_mmol_l (lab)',\n       'serum_albumin_g_dl (lab)'